In [58]:
import pandas as pd
import numpy as np

venues = pd.read_csv("venues.csv", encoding="latin-1")
venues_more = pd.read_csv("venues_more.csv",encoding="latin-1")
events = pd.read_csv("events.csv",encoding="latin-1")
events_more = pd.read_csv("events_more.csv", encoding="latin-1")

In [66]:
g_using = {}
for gid in events.group_id.unique():
    g_using[gid] = set()
for gid in events_more.group_id.unique():
    g_using[gid] = set()
for row in events.itertuples(index=True,name='Pandas'):
    g_using[getattr(row,'group_id')].add(getattr(row,'venue_id'))
for row in events_more.itertuples(index=True,name='Pandas'):
    g_using[getattr(row,'group_id')].add(getattr(row,'venue_id'))
    
venue_to_venue = {}
venue_adj_list = {}

for row in events.itertuples(index=True, name='Pandas'):
    for vid in g_using[getattr(row,'group_id')]:
        if vid != getattr(row,'venue_id'):
            if (getattr(row,'venue_id'),vid) not in venue_to_venue:
                venue_to_venue[(getattr(row,'venue_id'),vid)] = set()
            venue_to_venue[(getattr(row,'venue_id'),vid)].add(getattr(row,'group_id'))
            if (vid,getattr(row,'venue_id')) not in venue_to_venue:
                venue_to_venue[(vid,getattr(row,'venue_id'))] = set()
            venue_to_venue[(vid,getattr(row,'venue_id'))].add(getattr(row,'group_id'))
    if getattr(row,'venue_id') not in venue_adj_list:
        venue_adj_list[getattr(row,'venue_id')] = []
    venue_adj_list[getattr(row,'venue_id')].extend([x for x in list(g_using[getattr(row,'group_id')]) if x != getattr(row,'venue_id')])

for row in events_more.itertuples(index=True, name='Pandas'):
    for vid in g_using[getattr(row,'group_id')]:
        if vid != getattr(row,'venue_id'):
            if (getattr(row,'venue_id'),vid) not in venue_to_venue:
                venue_to_venue[(getattr(row,'venue_id'),vid)] = set()
            venue_to_venue[(getattr(row,'venue_id'),vid)].add(getattr(row,'group_id'))
            if (vid,getattr(row,'venue_id')) not in venue_to_venue:
                venue_to_venue[(vid,getattr(row,'venue_id'))] = set()
            venue_to_venue[(vid,getattr(row,'venue_id'))].add(getattr(row,'group_id'))
    if getattr(row,'venue_id') not in venue_adj_list:
        venue_adj_list[getattr(row,'venue_id')] = []
    venue_adj_list[getattr(row,'venue_id')].extend([x for x in list(g_using[getattr(row,'group_id')]) if x != getattr(row,'venue_id')])

502
12809


In [60]:
groups_topics = pd.read_csv('groups_topics.csv',encoding='latin-1')

In [61]:
def get_common_topics(grouplist):
    com = groups_topics[groups_topics['group_id'] == grouplist[0]].topic_id.unique()
    for x in grouplist[1:]:
        com = np.intersect1d(com,groups_topics[groups_topics['group_id'] == x].topic_id.unique())
    com = [groups_topics[groups_topics['topic_id'] == x].topic_name.unique()[0] for x in com]
    return com

In [62]:
len(venue_to_venue)

337522

In [ ]:
new_dict_venue_to_venue = {'From': [], 'To': [], 'Number of Shared Groups': [], 
                           'Shared Groups': [], 'Common Topics of Groups': []}
i = 0
for key, value in venue_to_venue.items():
    if i%10000 == 0:
        print(i)
    new_dict_venue_to_venue['From'].append(key[0])
    new_dict_venue_to_venue['To'].append(key[1])
    new_dict_venue_to_venue['Number of Shared Groups'].append(len(value))
    new_dict_venue_to_venue['Shared Groups'].append(list(value))
    new_dict_venue_to_venue['Common Topics of Groups'].append(get_common_topics(list(value)))
    i += 1

In [14]:
df = pd.DataFrame.from_dict(new_dict_venue_to_venue)

In [15]:
df.to_csv('venues_edges.csv')

In [76]:
ogids = venues['venue_id'].unique()
moreids = venues_more['venue_id'].unique()

In [78]:
node_df = pd.DataFrame(columns=['venue_id','venue_name','city'])
noti = 0
i = 0
for venue in venue_adj_list.keys():
    if i%100==0:
        print(i)
    i+=1
    if venue in ogids:
        row = venues[venues['venue_id'] == venue]
        node_df = node_df.append({'venue_id': venue,
                                 'venue_name': row['venue_name'].unique()[0],
                                 'city': row['city'].unique()[0]},ignore_index=True)
    elif venue in moreids:
        row = venues_more[venues_more['venue_id'] == venue]
        node_df = node_df.append({'venue_id': venue,
                                 'venue_name': row['venue_name'].unique()[0],
                                 'city': row['city'].unique()[0]},ignore_index=True)
    else:
        node_df = node_df.append({'venue_id': venue,
                                 'venue_name': 'unk',
                                 'city': 'unk'},ignore_index=True)
        noti += 1

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800


In [79]:
noti

84

In [82]:
node_df.head()

,venue_id,venue_name,city
0,23729697,Little Szechuan,San Francisco
1,16948982,Horizon Lounge,San Francisco
2,24717469,taste of vietnam noodle bar,San Francisco
3,724783,Horizon,San Francisco
4,20984572,Score! Bar and Lounge,San Francisco


In [80]:
node_df.to_csv('venues_nodes.csv')

In [81]:
len(df), len(node_df)

(337522, 12809)